In [ ]:
import sys, os, h5py, time, itertools
import numpy as np
from scipy import stats, optimize
from problem import Problem
from iminuit import Minuit

# Load global optimization results

We set up the fitting helper class as before and load the results which were obtained in Step 2 of the analysis.

In [ ]:
perseus = Problem('./Data/ReducedData/Perseus_Data.h5',
                  './Data/ModelComponents/APEC_Grid.npz',
                  './Data/ModelComponents/Absorption_Grid.npy',
                  './Fitting/Initial.npz',
                  min_energy =3, max_energy = 6,)

# Load the fit from step 2
model_fit = np.load('./Fitting/Perseus_Global_Fit.npy')

# Iteratively test and drop lines with a local optimizer

Here we define our threshold for inclusion for a line to be when the removal results in a $\Delta \chi^2 \geq 3$ so as to maximize the $\chi_\nu^2$.

In [ ]:
keep_threshold = 3

In [ ]:
def drop_test(index):
    start = time.time()
    if index in dropped_lines:
        return np.inf, np.inf, old

    drop_guess = np.copy(old)

    for i in np.append(dropped_lines, index):
    
        drop_guess[problem.num_continuum_params + i] = 0
        drop_guess[problem.num_continuum_params + problem.num_lines + i] = 0
        drop_guess[problem.num_continuum_params + 2*problem.num_lines + i] = 1e-3

    n = Minuit.from_array_func(problem.fitness, drop_guess, limit = problem.bounds.T, errordef = 1e0)

    for i in np.append(dropped_lines, index):

        n.fixed['x' + str(problem.num_continuum_params + i)] = True
        n.fixed['x' + str(problem.num_continuum_params + problem.num_lines + i)] = True
        n.fixed['x' + str(problem.num_continuum_params + 2*problem.num_lines + i)] = True

    n.migrad(ncall = int(1e8))
    eval_sig = n.fval - old_fitness_val
    end = time.time()
    print(index, end-start, eval_sig)
    return eval_sig, n.fval, n.np_values()

In [ ]:
init = np.copy(m.values.values())
init_fitness_val = m.fval

dropped_lines = np.zeros((0), dtype = np.int)
eval_sig = np.zeros_like(problem.all_lines)
chiSq_array = np.zeros_like(eval_sig)
fit_array = np.zeros((len(eval_sig), len(m.np_values())))

while True:
    for i in range(len(eval_sig)):
        eval_sig[i], chiSq_array[i], fit_array[i] = drop_test(i)

    if np.amin(eval_sig) < keep_threshinit:
        dropped_line = np.argmin(eval_sig)
        print('Dropping: ', dropped_line, problem.all_lines[dropped_line], chiSq_array[dropped_line])

        init_fitness_val = chiSq_array[dropped_line]
        init = fit_array[dropped_line]
        dropped_lines = np.append(dropped_lines, dropped_line)
    else:
        print('None to drop')
        break
    print()


# Validate the Line Dropping

In [ ]:
drop_guess = np.copy(init)

for i in dropped_lines:

    drop_guess[problem.num_continuum_params + i] = 0
    drop_guess[problem.num_continuum_params + problem.num_lines + i] = 0
    drop_guess[problem.num_continuum_params + 2 * problem.num_lines + i] = 0

n = Minuit.from_array_func(problem.fitness, drop_guess, limit=problem.bounds.T, errordef = 1e0)

for i in dropped_lines:

    n.fixed['x' + str(problem.num_continuum_params + i)] = True
    n.fixed['x' + str(problem.num_continuum_params + problem.num_lines + i)] = True
    n.fixed['x' + str(problem.num_continuum_params + 2 * problem.num_lines + i)] = True

n.migrad()

# Save a new model specification

In [ ]:
new_fit_path = './Fitting/DroppedLine.npz'

fit = np.array(n.values.values())
upper = problem.bounds[1, :]
lower = problem.bounds[0, :]

delete_locs = np.where(n.fixed.values())[0]

fit = np.delete(fit, delete_locs)
upper = np.delete(upper, delete_locs)
lower = np.delete(lower, delete_locs)

fit_dict = dict()
fit_dict['z'] = problem.z
fit_dict['num_apec'] = problem.num_apec
fit_dict['num_folded'] = problem.num_folded
fit_dict['num_unfolded'] = problem.num_unfolded
fit_dict['lines'] = np.delete(problem.all_lines, dropped_lines)
fit_dict['redshifts'] = np.delete(problem.redshifts, dropped_lines)

fit_dict['guess'] = fit
fit_dict['upper'] = upper
fit_dict['lower'] = lower

np.savez(new_fit_path, **fit_dict)

# Generate a new problem instance with the new model  

Here we use `keep_thresh = 2` because continuum model components are specified with two parameters.

In [ ]:
keep_threshold = 2

model_path = './Fitting/E3/Perseus/DroppedLine.npz'

perseus = Problem('./Data/ReducedData/Perseus_Data.h5',
                  './Data/ModelComponents/APEC_Grid.npz',
                  './Data/ModelComponents/Absorption_Grid.npy',
                  model_path,
                  min_energy =3, max_energy = 6,)

# Load the fit from step 2
model_fit = np.load(model_path)['guess']

In [ ]:
def drop_test(index):
    start = time.time()
    if index in dropped_continuum:
        return np.inf, np.inf, old
    
    drop_guess = np.copy(old)
    
    for i in np.append(dropped_continuum, index):
        
        if i < perseus.num_apec:

            drop_guess[i*2+1] = 0
            drop_guess[i*2+2] = 1
        else:
            drop_guess[i*2+1] = 0
            drop_guess[i*2+2] = 0
        
    n = Minuit.from_array_func(problem.fitness, drop_guess, limit = problem.bounds.T, errordef = 1e0)
    
    for i in np.append(dropped_continuum, index):
        n.fixed['x' + str(i*2+1)] = True
        n.fixed['x' + str(i*2+2)] = True
        
    n.migrad(ncall = int(1e8))
    eval_sig = n.fval - old_fitness_val
    end = time.time()
    print(index, end-start, eval_sig)
    return eval_sig, n.fval, np.array(n.values.values())

In [ ]:
init = np.copy(m.np_values())
init_fitness_val = m.fval

dropped_continuum = np.zeros((0), dtype = int)
eval_sig = np.zeros((4))
chiSq_array = np.zeros_like(eval_sig)
fit_array = np.zeros((len(eval_sig), len(m.np_values())))

while True:
    for i in range(len(eval_sig)):
        eval_sig[i], chiSq_array[i], fit_array[i] = drop_test(i)

    if np.amin(eval_sig) < keep_thresh:
        dropped_component = np.argmin(eval_sig)
        print('Dropping: ', dropped_component, chiSq_array[dropped_component])


        init_fitness_val = chiSq_array[dropped_component]
        init = fit_array[dropped_component]
        dropped_continuum = np.append(dropped_continuum, dropped_component)
    else:
        print('None to drop')
        break

# Validate the Continuum Dropping

In [ ]:
drop_guess = np.copy(old)

for i in dropped_continuum:
    
    
    if i == 0 or i == 1:
        drop_guess[i*2+1] = 0
        drop_guess[i*2+2] = 1
        
    else:
        drop_guess[i*2+1] = 0
        drop_guess[i*2+2] = 0

n = Minuit.from_array_func(problem.fitness, drop_guess, limit=problem.bounds.T, errordef = 1e0)

for i in dropped_continuum:
    n.fixed['x' + str(i*2+1)] = True
    n.fixed['x' + str(i*2+2)] = True


#n.strategy = 2
n.migrad()

# Save the Model Specification

Recall that in our analysis, we fix ther hydrogen depth at its best fit value at this stage and do not vary it further.

In [ ]:
new_fit_path = './Fitting/DroppedContinuum.npz'

fit = np.array(n.values.values())
upper = problem.bounds[1, :]
lower = problem.bounds[0, :]


delete_locs = np.where(n.fixed.values())[0]

fit = np.delete(fit, delete_locs)
upper = np.delete(upper, delete_locs)
lower = np.delete(lower, delete_locs)

fit_dict = dict()
fit_dict['z'] = problem.z

# The way we related the index of dropped components to type will vary depending on the
# model components that are specified. Are more elegant implementation could be made.
fit_dict['num_apec'] = problem.num_apec - (0 in dropped_continuum)- (1 in dropped_continuum)
fit_dict['num_folded'] = problem.num_folded - (2 in dropped_continuum)
fit_dict['num_unfolded'] = problem.num_unfolded-(3 in dropped_continuum)
fit_dict['lines'] = problem.all_lines
fit_dict['redshifts'] = problem.redshifts

fit_dict['guess'] = fit
fit_dict['upper'] = upper
fit_dict['lower'] = lower

np.savez(new_fit_path, **fit_dict)